In [2]:
import json
import os
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import text
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.text import tokenizer_from_json
import time as time

import pickle

Using TensorFlow backend.


In [3]:

reader = pd.read_csv('comments_csv',iterator=True, chunksize=10,lineterminator='\n' )
reader.get_chunk()

,Unnamed: 0,comment_id,comment,parent_comment_id,parent_post_id,num_angrys,num_hahas,num_loves,num_sads,num_thankfuls,num_wows,total_reaction,fb_page
0,0,10159788572690354_10159788622410354,You'll have to survive the mass hysteria and c...,10159788572690354_10159788608450354,95475020353_10159788572690354,0,0,0,0,0,0,8,brietbart
1,1,10155327674521130_10155327912376130,"Ummm...thanks. ""Not trying to be rude but"" is ...",10155327674521130_10155327709241130,18468761129_10155327674521130,0,0,1,0,0,0,37,huffpo
2,2,1234985053182879_1235051949842856,Get rid of the cap all together. Include all w...,1234985053182879_1235001863181198,18468761129_10153685924816130,0,0,0,0,0,0,9,huffpo
3,3,10156819866680354_1696714377242759,We will stay on the Trump train more than ever .,10156819866680354_10156819873650354,95475020353_10156819866680354,0,0,0,0,0,0,9,brietbart
4,4,10156242997995354_10156243051735354,Becca Portera But....but....Obamacare took car...,10156242997995354_10156243010900354,95475020353_10156242997995354,0,0,0,0,0,0,23,brietbart
5,5,10157901701015354_10157901768510354,"David"" Hell the Democrats control the Media n...",10157901701015354_10157901714205354,95475020353_10157901701015354,0,0,0,0,0,0,13,brietbart
6,6,10159620405955354_10159620498240354,Mike Daniels How so? Do you have a 401k? Hav...,10159620405955354_10159620415210354,95475020353_10159620405955354,0,0,0,0,0,0,9,brietbart
7,7,10159656230075354_10159656251530354,NaN,10159656230075354_10159656232315354,95475020353_10159656230075354,0,0,1,0,0,0,15,brietbart
8,8,10153245599621130_10153245817406130,Whenever I see a guy trolling I get the same u...,10153245599621130_10153245664966130,18468761129_10153245599621130,0,0,0,0,0,0,8,huffpo
9,9,10158434860435354_10158434922240354,Carlye Archibeque ACA is still around so who's...,10158434860435354_10158434867425354,95475020353_10158434860435354,0,0,0,0,0,0,16,brietbart


In [53]:

        

#group should be 'hp' or 'bb'
#rt should be 'lst' or 's'
def comments_gen(group, num, rt):
    hp_id = 18468761129 #fb group id for huffpo
    bb_id = 95475020353 #fb group id for brietbart
    
    if group == 'hp':
        group_id = hp_id
        group_col = 'huffpo'
    elif group =='bb':
        group_id = bb_id
        group_col = 'brietbart'
    else:
        print('invalid group')
        return
    
    reader = pd.read_csv('comments_csv',iterator=True, chunksize=num,lineterminator='\n' )
    
    for df_chunk in reader:
        single_group = df_chunk[df_chunk.fb_page==group_col].dropna()
        
        if rt == 's':
            yield single_group['comment']
        if rt == 'lst':
            yield single_group['comment'].tolist()

    

In [55]:
start = time.time()
TOP_N = 50000
bb_tok=Tokenizer(num_words=TOP_N, oov_token='<unk>' )

bb_tok.fit_on_texts(comments_gen(group='bb',num=1000, rt='lst'))

with open('bb-50k-tok.pickle', 'wb') as handle:
    pickle.dump(bb_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('bb tokenization time: {}'.format(time.time() - start))

hp_start = time.time()
hp_tok = Tokenizer(num_words=TOP_N, oov_token='<unk>')
hp_tok.fit_on_texts(comments_gen(group='hp',num=1000,rt='lst'))

with open('hp-50k-tok.pickle', 'wb') as handle:
    pickle.dump(hp_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('hp tokenization time: {}'.format(time.time() - hp_start ))


def just_comments_to_file(group, num): #group should be 'bb' or 'hp'
    for chunk in comments_gen(group, num, 's'):
        chunk.to_csv("{}_all_comments.csv".format(group), mode='a')

        
start = time.time()
just_comments_to_file('hp',2000)
just_comments_to_file('bb',2000)
print('elasped time: {}'.format(time.time() - start))

bb tokenization time: 231.6587655544281
hp tokenization time: 220.4470658302307


/home/dl-one/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


elasped time: 342.5961353778839


In [73]:

def just_comments_to_file(group, num): #group should be 'bb' or 'hp'
    for chunk in comments_gen(group, num, 's'):
        chunk.to_csv("{}_all_comments.csv".format(group), mode='a',header=['comment'])

        
start = time.time()
just_comments_to_file('hp',2000)
just_comments_to_file('bb',2000)
print('elasped time: {}'.format(time.time() - start))

elasped time: 337.34581756591797


In [75]:
all_bb = pd.read_csv('bb_all_comments.csv')
all_hp = pd.read_csv('hp_all_comments.csv')
print(len(all_bb))
print(len(all_hp))


7881969
7063461


In [76]:
all_bb.head().columns

Index(['0', 'You'll have to survive the mass hysteria and chaos first. People are going to come for what you have.'], dtype='object')

In [ ]:
reader = pd.read_csv('comments_csv',iterator=True, chunksize=1000,lineterminator='\n' )

start = time.time()
lst = [comment for comment in comment_itr(make_bb_comments()) ]
end = time.time()

print('elasped time to process {} samples: {}'.format(len(lst),end-start))

with open('bb_train_datav0.json', 'w') as outfile:
    json.dump(lst,outfile)

In [ ]:
with open("bb_train_datav0.json") as f:
    data=json.load(f)
    
less_20 = list(filter(lambda x: len(x) < 20, data))
with open('bb_train<20.json','w') as out:
    json.dump(less_20,out)